### Написать парсер страницы сайта https://mediakit.iportal.ru/our-team
На странице надо собрать данные и выгрузить в csv в формате Город, Имя, Должность, email.
Далее посчитать сколько фамилий сотрудников начинаются на каждую из букв русского алфавита (а - 10, б - 0 и т.д.)
Файлы с кодом  и результатом парсинга просим выслать на почту @brandpolgroup.com вместе с Вашим резюме. Удачи!


# Загружаем библиотеки

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

# Получаем soup страницы

In [2]:
URL = 'https://mediakit.iportal.ru/our-team/'
HEADERS = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}

data = requests.get(URL, headers=HEADERS)
print(data.status_code)
soup = BeautifulSoup(data.text, 'html.parser')

200


Смотрим, как выклядит контейнер с данными:

# Выделяем города

In [3]:
# города в soup под data-field-top-value=0 и =1
city_1 = soup.find('div', {'id': 'allrecords'}).findAll('div', 
                                                        {'data-elem-type': 'text',
                                                         'data-field-top-value': 1}
                                                       )
city_2 = soup.find('div', {'id': 'allrecords'}).findAll('div', 
                                                        {'data-elem-type': 'text',
                                                         'data-field-top-value': 0}
                                                       )
city_1+=city_2

# создаем список городов
city = []
for i in city_1:
    city.append(i.text.replace('\n', ''))  

# Выделяем имена

In [4]:
# имена в soup под data-field-top-value=200 и =209
name_1 = soup.find('div', {'id': 'allrecords'}).findAll('div', 
                                                        {'data-elem-type': 'text',
                                                         'data-field-top-value': 200}
                                                       )
name_2 = soup.find('div', {'id': 'allrecords'}).findAll('div', 
                                                        {'data-elem-type': 'text',
                                                         'data-field-top-value': 209}
                                                       )
name_1+=name_2

# создаем список имен
name = []
for i in name_1:
    name.append(i.text.replace('\n', ''))

# Выделяем должности

In [5]:
# работа в soup под data-field-top-value=237 и =245
job_1 = soup.find('div', {'id': 'allrecords'}).findAll('div', 
                                                        {'data-elem-type': 'text',
                                                         'data-field-top-value': 237}
                                                       )
job_2 = soup.find('div', {'id': 'allrecords'}).findAll('div', 
                                                        {'data-elem-type': 'text',
                                                         'data-field-top-value': 245}
                                                       )
job_1+=job_2

# создаем список работ
job = []
for i in job_1:
    job.append(''.join(i.text.replace('\n', '').split('8 800 2000-383, доб.')[:1]))
    


# Выделяем почту

In [6]:
mail = []
for i in job_1:
    
    try:
        mail.append(i.text.replace('\n', '').split('8 800 2000-383, доб.')[1][5:]) 
    except:
        mail.append(i.text.replace('\n', '').split('8 800 2000-383, доб.')[0][16:])

# Создаем датафрейм

In [7]:
df=pd.DataFrame()
df['city'] = pd.Series(city)
df['name'] = pd.Series(name)
df['job'] = pd.Series(job)
df['mail'] = pd.Series(mail)

In [8]:
df.head()

,city,name,job,mail
0,Москва,Андрей Затирко,Главный редактор,
1,Национальная редакция,Илья Ненко,Шеф-редакторrednews@iportal.ru,ews@iportal.ru
2,Екатеринбург,Артем Беседин,Главный редакторe1@iportal.ru,e1@iportal.ru
3,Новосибирск,Марина Малкова,Главный редакторnn@iportal.ru,nn@iportal.ru
4,Нижний Новгород,Денис Стрелков,Главный редактор74@iportal.ru,74@iportal.ru


# Сохраняем файл csv

In [9]:
df.to_csv('mediakit.csv', sep=',', encoding='utf-8')

# Посчитаем первые буквы фамилий

Для начала заменим NAN, что бы могли применить функции split() столбцу name

In [10]:
df = df.fillna('ь')

In [11]:
df.tail()

,city,name,job,mail
39,Чита,Екатерина Царёва,Менеджер по продажам,i.elkina@iportal.ru
40,Москва,Мария Ревина,ь,ь
41,SHKULEVMEDIAHOLDING,Оксана Маклакова,ь,ь
42,Ханты-Мансийск,ь,ь,ь
43,Красноярск,ь,ь,ь


Выделяем первые буквы фамилий

In [12]:
letter = []
for i in range(len(df['name'])):
    a = df['name'][i].split()[1][0]
    letter.append(a)


IndexError: list index out of range

Добаляем их в датафрейм

In [13]:
df['letter'] = pd.Series(letter)

Группируем по буквам

In [14]:
df.groupby(by=['letter'])['city'].count().sort_values(ascending=False)

letter
М    6
Б    6
К    5
З    3
И    3
Т    2
Г    2
Ц    2
П    2
Ф    2
С    2
Ё    1
Ш    1
Н    1
Р    1
Д    1
В    1
Ю    1
Name: city, dtype: int64

# Вывод:
    
    Получилось у нас, что шесть фамилий на букву М и Б, на букву К - 5 фамилий.